<a href="https://colab.research.google.com/github/nnilayy/Recurrent-Neural-Networks/blob/main/Fasta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import gc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

from keras.models import Sequential
from keras.layers import Dense
# measure roc auc score metric 
from tensorflow.keras.metrics import AUC

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
train_taxonomy=pd.read_csv("/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/Train/train_taxonomy.tsv", sep='\t')

In [16]:
train_taxonomy.insert(0,'TaxonomyID',train_taxonomy['taxonomyID'])
train_taxonomy.drop(columns='taxonomyID',inplace=True)

In [17]:
train_taxonomy

,TaxonomyID,EntryID
0,9606,Q8IXT2
1,559292,Q04418
2,7227,A8DYA3
3,284812,Q9UUI3
4,185431,Q57ZS4
...,...,...
142241,9606,Q5TD07
142242,7955,A8BB17
142243,7955,A0A2R8QBB1
142244,284812,P0CT72


In [ ]:
!pip install biopython

In [27]:
from Bio import SeqIO
records=[]
with open("/content/drive/MyDrive/CAFA_PROTEIN_FUNCTION/Train/train_sequences.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
      records.append(record)

In [28]:
sequences=[]
ids=[]
names=[]
descriptions=[]
dbxrefs=[]

for record in records:
  sequences.append(record.seq)
  ids.append(record.id)
  names.append(record.name)
  descriptions.append(record.description)
  dbxrefs.append(record.dbxrefs)

fasta_file={
    'Sequences':sequences,
    'IDs':ids,
    'names':names,
    'Descriptions':descriptions,
}
data=pd.DataFrame(fasta_file)
df=fasta=data

In [49]:
datas=fasta['Descriptions']
datas=pd.DataFrame(datas,columns=['Descriptions'])
datas=datas.reset_index()
datas.head()

,index,Descriptions
0,0,P20536 sp|P20536|UNG_VACCC Uracil-DNA glycosyl...
1,1,O73864 sp|O73864|WNT11_DANRE Protein Wnt-11 OS...
2,2,O95231 sp|O95231|VENTX_HUMAN Homeobox protein ...
3,3,A0A0B4J1F4 sp|A0A0B4J1F4|ARRD4_MOUSE Arrestin ...
4,4,P54366 sp|P54366|GSC_DROME Homeobox protein go...


## First DataFrame

In [50]:
import re
id_regex = r'(\S+)'
sp_id_regex = r'([a-zA-Z]{2}\|[^\|]+\|)'
# r'sp\|([^|]+)\|'

desc_regex =r'([^=]+)'  
os_regex = r'OS=([^=]+)'
ox_regex = r'OX=([^=]+)'
gn_regex = r'GN=([^=]+)'
pe_regex = r'PE=(\d+)'
sv_regex = r'SV=(\d+)'

columns = []
indexes=[]
for i in range(len(data)):
  line = datas['Descriptions'][i]
  match = re.match(fr'{id_regex} {sp_id_regex}{desc_regex} {os_regex} {ox_regex} {gn_regex} {pe_regex} {sv_regex}', line)
  if match:
    columns.append(match.groups())
    indexes.append(datas['index'][i])
    

df1=pd.DataFrame(columns,columns=['EntryID','SP_Entry_id','Description','OS','OX','GN','PE','SV'])
df1.insert(0, 'Index', indexes)
df1

,Index,EntryID,SP_Entry_id,Description,OS,OX,GN,PE,SV
0,0,P20536,sp|P20536|,UNG_VACCC Uracil-DNA glycosylase,Vaccinia virus (strain Copenhagen),10249,UNG,1,1
1,1,O73864,sp|O73864|,WNT11_DANRE Protein Wnt-11,Danio rerio,7955,wnt11,2,1
2,2,O95231,sp|O95231|,VENTX_HUMAN Homeobox protein VENTX,Homo sapiens,9606,VENTX,1,1
3,3,A0A0B4J1F4,sp|A0A0B4J1F4|,ARRD4_MOUSE Arrestin domain-containing protein 4,Mus musculus,10090,Arrdc4,1,1
4,4,P54366,sp|P54366|,GSC_DROME Homeobox protein goosecoid,Drosophila melanogaster,7227,Gsc,2,2
...,...,...,...,...,...,...,...,...,...
138834,142241,A0A286YAI0,tr|A0A286YAI0|,A0A286YAI0_DANRE Macrophage receptor with coll...,Danio rerio,7955,marco,4,1
138835,142242,A0A1D5NUC4,tr|A0A1D5NUC4|,A0A1D5NUC4_CHICK Mitogen-activated protein kin...,Gallus gallus,9031,MAP3K7,3,1
138836,142243,Q5RGB0,tr|Q5RGB0|,"Q5RGB0_DANRE Potassium channel, subfamily K, m...",Danio rerio,7955,kcnk5b,3,1
138837,142244,A0A2R8QMZ5,tr|A0A2R8QMZ5|,A0A2R8QMZ5_DANRE Myocyte enhancer factor 2aa,Danio rerio,7955,mef2aa,4,1


## Second DataFrame

In [51]:
missing=datas[datas['index'].isin(df1['Index'])==False]
indexes=missing['index']
missing.head()

,index,Descriptions
88,88,O76745 sp|O76745|NP_CIMLE Nitrophorin Cim l NP...
95,95,A0A193H396 sp|A0A193H396|CZS6_CRUCA Cruziosept...
153,153,P22241 sp|P22241|DRPD_CRAPL Desiccation-relate...
198,198,A0A6G9KJK9 sp|A0A6G9KJK9|TX10A_MANRB U10-myrmi...
207,207,Q04616 sp|Q04616|3O1D_RHOOP 3-oxosteroid 1-deh...


In [ ]:
import re
id_regex = r'(\S+)'
sp_id_regex = r'([a-zA-Z]{2}\|[^\|]+\|)'
desc_regex =r'([^=]+)'  
os_regex = r'OS=([^=]+)'
ox_regex = r'OX=([^=]+)'
pe_regex = r'PE=(\d+)'
sv_regex = r'SV=(\d+)'

columns = []
missing_indexes=[]
for i in indexes: 
  line = missing['Descriptions'][i]
  match = re.match(fr'{id_regex} {sp_id_regex}{desc_regex} {os_regex} {ox_regex} {pe_regex} {sv_regex}', line)
  if match:
    columns.append(match.groups())
    missing_indexes.append(missing['index'][i])
    
df2=pd.DataFrame(columns,columns=['EntryID','SP_Entry_id','Description','OS','OX','PE','SV'])
df2.insert(0, 'Index', missing_indexes)
df2.insert(6, 'GN',np.nan)
df2

## Third DataFrame

In [ ]:
missing_2=missing[missing['index'].isin(df2['Index'])==False]
indexes=missing_2['index']
missing_2

In [91]:
first='Q9PSQ5 tr|Q9PSQ5|Q9PSQ5_CHICK p60=60 kDa stress-related protein OS=Gallus gallus OX=9031 GN=0 PE=4 SV=1'
second='R9RWQ9 tr|R9RWQ9|R9RWQ9_BACIU Rap phosphatase OS=Bacillus subtilis subsp. subtilis NCIB 3610 = ATCC 6051 = DSM 10 OX=535026 GN=rapP PE=4 SV=1'
third='Q9PSP7 tr|Q9PSP7|Q9PSP7_CHICK Heterogeneous nuclear ribonucleoprotein U, MARS binding protein P120, hnRNP U=MATRIX attachment region binding protein OS=Gallus gallus OX=9031 GN=0 PE=1 SV=1'
fourth='Q5QP01 tr|Q5QP01|Q5QP01_HUMAN 3 beta-hydroxysteroid dehydrogenase/Delta 5-- OS=0 OX=0 GN=0 PE=0 SV=0'
fifth='B0QYD3 tr|B0QYD3|B0QYD3_HUMAN DNA dC- OS=0 OX=0 GN=0 PE=0 SV=0'
sixth='A0A087WZI3 tr|A0A087WZI3|A0A087WZI3_HUMAN DNA dC- OS=0 OX=0 GN=0 PE=0 SV=0'
data_missing=[first,second,third,fourth,fifth,sixth]

In [ ]:
import re
id_regex = r'(\S+)'
sp_id_regex = r'([a-zA-Z]{2}\|[^\|]+\|)'
desc_regex =r'([^=]+)'  
os_regex = r'OS=([^=]+)'
ox_regex = r'OX=([^=]+)'
pe_regex = r'PE=(\d+)'
sv_regex = r'SV=(\d+)'

columns = []
missing_indexes=[]
for i,j in zip(data_missing,indexes): 
  line = i
  match = re.match(fr'{id_regex} {sp_id_regex}{desc_regex} {os_regex} {ox_regex} {gn_regex} {pe_regex} {sv_regex}', line)
  if match:
    columns.append(match.groups())
    missing_indexes.append(indexes[j])
    
df3=pd.DataFrame(columns,columns=['EntryID','SP_Entry_id','Description','OS','OX','GN','PE','SV'])
df3.insert(0, 'Index', missing_indexes)
df3['OS']=np.nan
df3['OX']=np.nan
df3['GN']=np.nan
df3['PE']=np.nan
df3['SV']=np.nan
df3

## Fourth Dataframe


In [ ]:
data = [
    [97374,'Q9PSQ5', 'tr|Q9PSQ5|','Q9PSQ5_CHICK p60=60 kDa stress-related protein','Gallus gallus','9031', '0', '4', '1'], 
    [99528,'R9RWQ9', 'tr|R9RWQ9|','R9RWQ9_BACIU Rap phosphatase','Bacillus subtilis subsp. subtilis NCIB 3610 = ATCC 6051 = DSM 10','535026','rapP','4','1'], 
    [103062,'Q9PSP7', 'tr|Q9PSP7|','Q9PSP7_CHICK Heterogeneous nuclear ribonucleoprotein U, MARS binding protein P120, hnRNP U=MATRIX attachment region binding protein','Gallus gallus','9031', '0', '1', '1']
    ]

df4 = pd.DataFrame(data, columns=['Index','EntryID','SP_Entry_id','Description','OS','OX','GN','PE','SV'])
df4

## Combined DataFrame


In [112]:
combined_df = pd.concat([df1,df2,df3,df4], axis=0)
combined_df=combined_df.sort_values('Index')
combined_df=combined_df.set_index('Index')
combined_df

,EntryID,SP_Entry_id,Description,OS,OX,GN,PE,SV
Index,,,,,,,,
0,P20536,sp|P20536|,UNG_VACCC Uracil-DNA glycosylase,Vaccinia virus (strain Copenhagen),10249,UNG,1,1
1,O73864,sp|O73864|,WNT11_DANRE Protein Wnt-11,Danio rerio,7955,wnt11,2,1
2,O95231,sp|O95231|,VENTX_HUMAN Homeobox protein VENTX,Homo sapiens,9606,VENTX,1,1
3,A0A0B4J1F4,sp|A0A0B4J1F4|,ARRD4_MOUSE Arrestin domain-containing protein 4,Mus musculus,10090,Arrdc4,1,1
4,P54366,sp|P54366|,GSC_DROME Homeobox protein goosecoid,Drosophila melanogaster,7227,Gsc,2,2
...,...,...,...,...,...,...,...,...
142241,A0A286YAI0,tr|A0A286YAI0|,A0A286YAI0_DANRE Macrophage receptor with coll...,Danio rerio,7955,marco,4,1
142242,A0A1D5NUC4,tr|A0A1D5NUC4|,A0A1D5NUC4_CHICK Mitogen-activated protein kin...,Gallus gallus,9031,MAP3K7,3,1
142243,Q5RGB0,tr|Q5RGB0|,"Q5RGB0_DANRE Potassium channel, subfamily K, m...",Danio rerio,7955,kcnk5b,3,1


In [118]:
combined_df=combined_df.sort_values('EntryID')
combined_df['SP_Entry_id']=combined_df['SP_Entry_id'].str.split('|').str[0]
combined_df

,EntryID,SP_Entry_id,Description,OS,OX,GN,PE,SV
Index,,,,,,,,
81407,A0A009IHW8,sp,ABTIR_ACIB9 NAD(+) hydrolase AbTIR,Acinetobacter baumannii (strain 1295743),1310613,J512_3302,1,1
139244,A0A021WW32,tr,"A0A021WW32_DROME Verthandi, isoform B",Drosophila melanogaster,7227,vtd,1,1
84490,A0A021WZA4,tr,"A0A021WZA4_DROME Zydeco, isoform G",Drosophila melanogaster,7227,zyd,3,1
3773,A0A023FBW4,sp,E1142_AMBCJ Evasin P1142,Amblyomma cajennense,34607,NaN,3,1
9948,A0A023FBW7,sp,EV546_AMBCJ Evasin P546,Amblyomma cajennense,34607,NaN,3,1
...,...,...,...,...,...,...,...,...
125535,X6RKS3,tr,X6RKS3_HUMAN Major prion protein (Fragment),Homo sapiens,9606,PRNP,1,1
99974,X6RLN4,tr,X6RLN4_HUMAN La-related protein 4 (Fragment),Homo sapiens,9606,LARP4,1,1
114434,X6RLP6,tr,X6RLP6_HUMAN RNA-binding region-containing pro...,Homo sapiens,9606,RNPC3,1,1


In [141]:
total_sum=combined_df['OS'].value_counts().sum()
df69=pd.DataFrame(combined_df['OS'].value_counts()/total_sum*100)
df69['OS'][:250].sum()

95.14352200108266

In [136]:
pd.DataFrame(combined_df['OS'].unique())

,0
0,Acinetobacter baumannii (strain 1295743)
1,Drosophila melanogaster
2,Amblyomma cajennense
3,Amblyomma parvum
4,Amblyomma triste
...,...
3152,Plasmodium falciparum (isolate 7G8)
3153,Porcine coronavirus HKU15
3154,Cynoglossus semilaevis
3155,Hirudo nipponia


In [119]:
combined_df.to_csv('hi.csv')